<a href="https://colab.research.google.com/github/lliyuan/CustomizedChatBot/blob/main/Lenny_Chatbot!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Lenny Chatbot Colab!

This Colab notebook contains all of the code you need to make a basic chatbot that will answer questions about a corpus of text. Colab is a cloud-based programming environment which will let you run all of this code from your browser.

At each step, follow the written instructions and press the "play" button next to the code sample in order to run it.

**Important Note:** This is a basic chatbot running on a limited selection of articles. It's only a starting point to show you what's possible!

If you have questions, feel free to reach out to me on Twitter at [@danshipper](https://www.twitter.com/danshipper).

## 1. Download our text corpus

The first thing we need to do is download the text our chatbot is going to use as reference material for answering questions.

In the Lenny Chatbot, I used every article he's written as the text corpus. But for this public codebase, I've collected two articles from his archive that we can use as a starting point.

These are the articles I'm using:

- [What is good retention?](https://www.lennysnewsletter.com/p/what-is-good-retention-issue-29)
- [How the biggest consumer apps got their first 1,000 users
](https://www.lennysnewsletter.com/p/how-the-biggest-consumer-apps-got)

You can replace these articles with any text corpus you want, however.


In [ ]:
! git clone https://github.com/EveryInc/Lenny-Newsletter-Corpus

Cloning into 'Lenny-Newsletter-Corpus'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (9/9), 43.31 KiB | 7.22 MiB/s, done.


In [ ]:
! ls -al

total 20
drwxr-xr-x 1 root root 4096 Apr 23 02:59 .
drwxr-xr-x 1 root root 4096 Apr 23 02:59 ..
drwxr-xr-x 4 root root 4096 Apr 19 13:36 .config
drwxr-xr-x 3 root root 4096 Apr 23 02:59 Lenny-Newsletter-Corpus
drwxr-xr-x 1 root root 4096 Apr 19 13:37 sample_data


# 2. Install our dependencies and define our functions

In this section we'll install GPT Index and Langchain. We'll also define the functions that we'll use later to construct our index and query it.

First, let's install our dependencies.

In [ ]:
!pip install gpt-index
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.8/548.8 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Now, we'll define the functions we're going to use later in order to construct our index and query it.

In [ ]:
from gpt_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 256
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    
    # it will use OpenAI embedding API to generate the embeddings
    # https://platform.openai.com/docs/guides/embeddings/embeddings
    index = GPTSimpleVectorIndex.from_documents(
        documents, service_context=service_context
    )

    index.save_to_disk('index.json')

    return index

def ask_lenny():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("What do you want to ask Lenny? ")
        response = index.query(query, response_mode="compact")
        display(Markdown(f"Lenny Bot says: <b>{response.response}</b>"))
  

# 3. Set OpenAI API Key
In order to run this notebook you'll need an API key from OpenAI. 

If you don't have one already, you can grab one by [signing up](https://platform.openai.com/overview). Then click your account icon on the top right of the screen and select "View API Keys". Create an API key.

Then run the code below and paste it into the text input.



In [ ]:
os.environ["OPENAI_API_KEY"] = input("enter your OpenAI API Key")

# 4. Construct Index

Now we're going to construct our index. This will take every file in the folder 'Lenny-Newsletter-Corpus', split it into chunks, and embed it with OpenAI's embeddings API.

**Important Note:** This step costs money. Running it on the text corpus we've given you by default should only cost $0.03 in total. But if you use other pieces of text be careful if they're really long.


In [ ]:
construct_index('/content/Lenny-Newsletter-Corpus')

# 5. Ask Questions!

Now we'll run the "ask_lenny" function we defined above. 

This will prompt the you to input a question, and then it will find chunks of text that might answer the question, and summarize the answer from those text chunks using GPT-3.

Remember, in this public Colab file we're only using two of Lenny's articles for our corpus. So it will only answer questions from:

- [What is good retention?](https://www.lennysnewsletter.com/p/what-is-good-retention-issue-29)
- [How the biggest consumer apps got their first 1,000 users
](https://www.lennysnewsletter.com/p/how-the-biggest-consumer-apps-got)


A few sample questions you can ask:

- What is good retention for a consumer social product?

- How did DoorDash get its first users?

- How did LinkedIn get started?

Again, this step costs money. So be aware!

In [ ]:
ask_lenny()

What do you want to ask Lenny? How did LinkedIn get started?


Lenny Bot says: <b>

LinkedIn got started in 2003 when Reid Hoffman, a former executive at PayPal, launched the professional networking platform. He intentionally seeded the product with successful friends and connections, recognizing that cultivating an aspirational brand was crucial to drive mainstream adoption.</b>

KeyboardInterrupt: ignored